# In Depth A/B Testing - Lab

## Introduction

In this lab, you'll explore a survey from Kaggle regarding budding data scientists. With this, you'll form some initial hypotheses, and test them using the tools you've acquired to date. 

## Objectives

You will be able to:
* Conduct t-tests and an ANOVA on a real-world dataset and interpret the results

## Load the Dataset and Perform a Brief Exploration

The data is stored in a file called **multipleChoiceResponses_cleaned.csv**. Feel free to check out the original dataset referenced at the bottom of this lab, although this cleaned version will undoubtedly be easier to work with. Additionally, meta-data regarding the questions is stored in a file name **schema.csv**. Load in the data itself as a Pandas DataFrame, and take a moment to briefly get acquainted with it.

> Note: If you can't get the file to load properly, try changing the encoding format as in `encoding='latin1'`

In [20]:
#Your code here
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv('multipleChoiceResponses_cleaned.csv', encoding='latin1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26394 entries, 0 to 26393
Columns: 230 entries, GenderSelect to AdjustedCompensation
dtypes: float64(15), object(215)
memory usage: 46.3+ MB


In [3]:
df.head(10)

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
0,"Non-binary, genderqueer, or gender non-conforming",NaN,NaN,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Female,United States,30.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Somewhat important,NaN,NaN,NaN,NaN
2,Male,Canada,28.0,"Not employed, but looking for work",NaN,NaN,NaN,NaN,NaN,NaN,...,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,Very Important,NaN,NaN
3,Male,United States,56.0,"Independent contractor, freelancer, or self-em...",NaN,NaN,Yes,NaN,Operations Research Practitioner,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,250000.0
4,Male,Taiwan,38.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Male,Brazil,46.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Male,United States,35.0,Employed full-time,NaN,NaN,Yes,NaN,Computer Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Female,India,22.0,Employed full-time,NaN,NaN,No,Yes,Software Developer/Software Engineer,Fine,...,Very Important,Somewhat important,Somewhat important,Not important,Very Important,Very Important,Somewhat important,Somewhat important,NaN,NaN
8,Female,Australia,43.0,Employed full-time,NaN,NaN,Yes,NaN,Business Analyst,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.802310,64184.8
9,Male,Russia,33.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017402,20882.4


## Wages and Education

You've been asked to determine whether education is impactful to salary. Develop a hypothesis test to compare the salaries of those with Master's degrees to those with Bachelor's degrees. Are the two statistically different according to your results?

> Note: The relevant features are stored in the 'FormalEducation' and 'AdjustedCompensation' features.

You may import the functions stored in the `flatiron_stats.py` file to help perform your hypothesis tests. It contains the stats functions that you previously coded: `welch_t(a,b)`, `welch_df(a, b)`, and `p_value(a, b, two_sided=False)`. 

Note that `scipy.stats.ttest_ind(a, b, equal_var=False)` performs a two-sided Welch's t-test and that p-values derived from two-sided tests are two times the p-values derived from one-sided tests. See the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) for more information.    

In [21]:
#Your code here
df['FormalEducation'].value_counts()

Master's degree                                                      8204
Bachelor's degree                                                    4811
Doctoral degree                                                      3543
Some college/university study without earning a bachelor's degree     786
Professional degree                                                   451
I did not complete any formal education past high school              257
I prefer not to answer                                                 90
Name: FormalEducation, dtype: int64

In [6]:
#Created a null and alternative hypothesis
'''
Null Hypothesis: There is no significant difference between salaries of data scientists with master's degree education and those with bachelor's education.

Alternative Hypothesis: There is a significant difference in salaries between data scientists with master's degree education and those with bachelor's education.

'''


"\nNull Hypothesis: There is no significant difference between salaries of data scientists with master's degree education and those with bachelor's education.\n\nAlternative Hypothesis: There is a significant difference in salaries between data scientists with master's degree education and those with bachelor's education.\n\n"

In [22]:
#Remove all null values under masters & bachelor's degrees
masters = df[df["FormalEducation"]== "Master's degree"]
cleaned_masters = masters.AdjustedCompensation
cleaned_masters = cleaned_masters.dropna()
cleaned_masters

3        250000.000
14        36634.400
27        53352.000
31        35419.104
37        80000.000
            ...    
26148     54670.000
26159         1.000
26180     65770.430
26255     89686.950
26378     10000.000
Name: AdjustedCompensation, Length: 1990, dtype: float64

In [23]:
bachelors = df[df["FormalEducation"]== "Bachelor's degree"]
cleaned_bachelor = bachelors.AdjustedCompensation
cleaned_bachelor = cleaned_bachelor.dropna()
cleaned_bachelor

8         64184.800
9         20882.400
11         1483.900
21        20000.000
25        10858.848
            ...    
26031     39050.000
26072     31878.000
26101      3336.000
26185     50000.000
26195    100449.384
Name: AdjustedCompensation, Length: 1107, dtype: float64

In [26]:
#significance level = 0.05
from scipy import stats
stats.ttest_ind(cleaned_masters,cleaned_bachelor, equal_var=False)


Ttest_indResult(statistic=0.43786693335411514, pvalue=0.6615527890254489)

In [27]:
#The p-value

p_value = 0.6615527890254489/2
print(p_value)

0.33077639451272445


In [30]:
cleaned_masters_mean = cleaned_masters.mean()
print("cleaned_masters_mean:", cleaned_masters_mean)
cleaned_bachelor_mean = cleaned_bachelor.mean()
print("cleaned_bachelor_mean:", cleaned_bachelor_mean)

cleaned_masters_median = cleaned_masters.median()
print("cleaned_masters_median:", cleaned_masters_median)
cleaned_bachelor_median = cleaned_bachelor.median()
print("cleaned_bachelor_median:", cleaned_bachelor_median)

sample_size_cleaned_masters = len(cleaned_masters)
print("sample_size_cleaned_masters:", sample_size_cleaned_masters)
sample_size_cleaned_bachelor = len(cleaned_bachelor)
print("sample_size_cleaned_bachelor:", sample_size_cleaned_bachelor)

print("welch t-test pvalue:",p_value)

cleaned_masters_mean: 69139.89987120005
cleaned_bachelor_mean: 64887.09799461879
cleaned_masters_median: 53812.170000000006
cleaned_bachelor_median: 38399.4
sample_size_cleaned_masters: 1990
sample_size_cleaned_bachelor: 1107
welch t-test pvalue: 0.33077639451272445


In [29]:
"""
The pvalue result turne out to be higher than the significance level (0.05). 
Therefore, we fail to reject the null hypothesis.

Conclusion: There is no significant difference between salaries of 
data scientists with master's degree education and those with bachelor's education.
"""

"\nThe pvalue result turne out to be higher than the significance level (0.05). \nTherefore, we fail to reject the null hypothesis.\n\nConclusion: There is no significant difference between salaries of \ndata scientists with master's degree education and those with bachelor's education.\n"

## Wages and Education II

Now perform a similar statistical test comparing the AdjustedCompensation of those with Bachelor's degrees and those with Doctorates. If you haven't already, be sure to explore the distribution of the AdjustedCompensation feature for any anomalies. 

In [32]:
#Your code here
doctoral = df[df["FormalEducation"]== "Doctoral degree"]
doctoral_clean = doctoral.AdjustedCompensation
doctoral_clean = doctoral_clean.dropna()

bachelors = df[df["FormalEducation"]== "Bachelor's degree"]
bachelors_clean= bachelors.AdjustedCompensation
bachelors_clean = bachelors_clean.dropna()
doctoral_clean.median(), bachelors_clean.median()

(74131.91999999998, 38399.4)

In [34]:
stats.ttest_ind(doctoral_clean, bachelors_clean, equal_var=False)

Ttest_indResult(statistic=1.0081234695549772, pvalue=0.31364763989440514)

In [35]:
p_value = 0.31364763989440514/2
p_value

0.15682381994720257

In [36]:
doctoral_clean_median = doctoral_clean.median()
print("doctoral_clean_median:", doctoral_clean_median)
bachelors_clean_median = bachelors_clean.median()
print("bachelors_clean_median:", bachelors_clean_median)

doctoral_clean_mean = doctoral_clean.mean()
print("doctoral_clean_mean:", doctoral_clean_mean)
bachelors_clean_mean = bachelors_clean.mean()
print("bachelors_clean_mean:", bachelors_clean_mean)

sample_doctoral_clean = len(doctoral_clean)
print("sample_doctoral_clean:", sample_doctoral_clean)
sample_bachelors_clean = len(bachelors_clean)
print("sample_bachelors_clean:", sample_bachelors_clean)

print("welch t-test pvalue:",p_value)

doctoral_clean_median: 74131.91999999998
bachelors_clean_median: 38399.4
doctoral_clean_mean: 29566175.76245311
bachelors_clean_mean: 64887.09799461879
sample_doctoral_clean: 967
sample_bachelors_clean: 1107
welch t-test pvalue: 0.15682381994720257


In [40]:
doctoral_clean.describe()

count    9.670000e+02
mean     2.956618e+07
std      9.099981e+08
min     -7.351631e+01
25%      4.000000e+04
50%      7.413192e+04
75%      1.200000e+05
max      2.829740e+10
Name: AdjustedCompensation, dtype: float64

In [41]:
#finding outliers using interquantile method
Q1 = doctoral_clean.quantile(0.25)
Q3 = doctoral_clean.quantile(0.75)
IQR = Q3 - Q1
upper_limit = Q3 + 1.5 * IQR
lower_limit = Q1 - 1.5 * IQR
upper_limit, lower_limit

(240000.0, -80000.0)

In [42]:
new_df = df[(df.AdjustedCompensation < upper_limit) & (df.AdjustedCompensation > lower_limit)]
new_df

,GenderSelect,Country,Age,EmploymentStatus,StudentStatus,LearningDataScience,CodeWriter,CareerSwitcher,CurrentJobTitleSelect,TitleFit,...,JobFactorTitle,JobFactorCompanyFunding,JobFactorImpact,JobFactorRemote,JobFactorIndustry,JobFactorLeaderReputation,JobFactorDiversity,JobFactorPublishingOpportunity,exchangeRate,AdjustedCompensation
8,Female,Australia,43.0,Employed full-time,NaN,NaN,Yes,NaN,Business Analyst,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.802310,64184.800
9,Male,Russia,33.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017402,20882.400
11,Male,India,27.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015620,1483.900
14,Male,Taiwan,26.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.033304,36634.400
21,Male,United States,25.0,Employed part-time,NaN,NaN,Yes,NaN,Researcher,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,20000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26185,Male,Other,56.0,Employed full-time,NaN,NaN,Yes,NaN,DBA/Database Engineer,Poorly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,50000.000
26195,Male,Germany,34.0,Employed full-time,NaN,NaN,Yes,NaN,Software Developer/Software Engineer,Fine,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.195826,100449.384
26203,Male,United States,50.0,Employed full-time,NaN,NaN,Yes,NaN,Data Scientist,Perfectly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,200000.000
26255,Male,Norway,32.0,Employed full-time,NaN,NaN,Yes,NaN,Data Analyst,Perfectly,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.195826,89686.950


In [43]:
doc = new_df[new_df["FormalEducation"]== "Doctoral degree"]
new_doctoral_clean = doc.AdjustedCompensation
new_doctoral_clean = new_doctoral_clean.dropna()


/tmp/ipykernel_134095/2966297125.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bach = new_df[df["FormalEducation"]== "Bachelor's degree"]


In [44]:
bach = new_df[df["FormalEducation"]== "Bachelor's degree"]
new_bachelors_clean= bach.AdjustedCompensation
new_bachelors_clean = new_bachelors_clean.dropna()

/tmp/ipykernel_134095/843627693.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  bach = new_df[df["FormalEducation"]== "Bachelor's degree"]


In [46]:
stats.ttest_ind(new_doctoral_clean, new_bachelors_clean, equal_var=False)

Ttest_indResult(statistic=12.381896214999609, pvalue=6.483887013753704e-34)

In [48]:
new_p_value = 6.483887013753704e-34/2
new_p_value

3.241943506876852e-34

In [50]:
sample_one = len(new_doctoral_clean)
print("new_doctoral_clean sample size:", sample_one)
sample_two = len(new_bachelors_clean)
print("new_bachelors_clean sample size:", sample_two)

new_doctoral_clean sample size: 934
new_bachelors_clean sample size: 1093


## Wages and Education III

Remember the multiple comparisons problem; rather than continuing on like this, perform an ANOVA test between the various 'FormalEducation' categories and their relation to 'AdjustedCompensation'.

In [52]:
#Your code here
import statsmodels.api as sm
from statsmodels.formula.api import ols
x= 'FormalEducation'
y= 'AdjustedCompensation'

formula = '{} ~ C({})'.format(y, x)
lm = ols(formula, df).fit()
table = sm.stats.anova_lm(lm, typ=2)
print(table)

                          sum_sq      df         F    PR(>F)
C(FormalEducation)  6.540294e+17     6.0  0.590714  0.738044
Residual            7.999414e+20  4335.0       NaN       NaN


## Additional Resources

Here's the original source where the data was taken from:  
    [Kaggle Machine Learning & Data Science Survey 2017](https://www.kaggle.com/kaggle/kaggle-survey-2017)

## Summary

In this lab, you practiced conducting actual hypothesis tests on actual data. From this, you saw how dependent results can be on the initial problem formulation, including preprocessing!